In [1]:
import xarray as xr
import uuid
from datetime import datetime as dt
import numpy as np

# Creating our data

Let's imagine we have an ice core that is 80 cm long. We have split the ice core into 8 10 cm chunks, and melted each one. We have then measured the salinity for each 10 cm chunk. Each salinity is therefore not representative of a single depth, but instead a 10 cm depth range - a cell.

In [2]:
sea_ice_salinity = np.array([35.5, 35.6, 35.8, 36, 36.2, 35.9, 35.8, 35.4])
depth_bounds = np.array([[0, 10],[10,20],[20,30],[30,40],[40,50],[50,60],[60,70],[70,80]])

There are 2 depths (a 2D array) associated with each salinity value - the minimum and maximum depth point for each chunk. However, we have a 1D array of salinity values. We therefore need a single depth to relate to each salinity value. We can take the mid depth for each ice chunk.

In [3]:
mid_depth = [np.mean(d) for d in depth_bounds]
print(mid_depth)

[5.0, 15.0, 25.0, 35.0, 45.0, 55.0, 65.0, 75.0]


# Initialising our file and creating dimensions

We are now going to initialise an xarray object, that will later become our NetCDF file. In the process, we are adding data to it too.

In [4]:
xrds = xr.Dataset(
    coords = {
        'depth': mid_depth
    },
    data_vars = {
        'sea_ice_salinity': ("depth", sea_ice_salinity)
    }
)
print(xrds)

<xarray.Dataset>
Dimensions:           (depth: 8)
Coordinates:
  * depth             (depth) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75.0
Data variables:
    sea_ice_salinity  (depth) float64 35.5 35.6 35.8 36.0 36.2 35.9 35.8 35.4


But what about our depth bounds? Let's add a dimension for them now.

In [5]:
# Add a new dimensions called depth_bounds, with 2 points
xrds = xrds.assign_coords(bounds =[0,1])
print(xrds)

<xarray.Dataset>
Dimensions:           (depth: 8, bounds: 2)
Coordinates:
  * depth             (depth) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75.0
  * bounds            (bounds) int64 0 1
Data variables:
    sea_ice_salinity  (depth) float64 35.5 35.6 35.8 36.0 36.2 35.9 35.8 35.4


Now we need to add a variable for the depth_bounds

In [6]:
xrds = xrds.assign(depth_bounds = (['depth','bounds'], depth_bounds))
print(xrds)

<xarray.Dataset>
Dimensions:           (depth: 8, bounds: 2)
Coordinates:
  * depth             (depth) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75.0
  * bounds            (bounds) int64 0 1
Data variables:
    sea_ice_salinity  (depth) float64 35.5 35.6 35.8 36.0 36.2 35.9 35.8 35.4
    depth_bounds      (depth, bounds) int64 0 10 10 20 20 30 ... 60 60 70 70 80


# Variable attributes

So now we have all of our data in our file. However, variable attributes must be added to make each variable understandable.

In [7]:
xrds['depth'].attrs = {
    'standard_name': 'depth',
    'long_name':'depth from top of sea ice',
    'units': 'cm',
    'coverage_content_type': 'coordinate',
    'positive': 'down',
    'bounds': 'depth_bounds' # See here we refer to the variable that contains the bounds for this coordinate variable
}

xrds['depth_bounds'].attrs = {
    'long_name':'Minimum and maximum depth bounds in sea ice core',
    'units': 'cm'
}

xrds['sea_ice_salinity'].attrs = {
    'standard_name': 'sea_ice_salinity',
    'long_name': 'Practical salinity of sea ice',
    'units': '1e-3',
    'coverage_content_type': 'physicalMeasurement'
    }

print(xrds['depth'], '\n')
print(xrds['depth_bounds'], '\n')
print(xrds['sea_ice_salinity'], '\n')


<xarray.DataArray 'depth' (depth: 8)>
array([ 5., 15., 25., 35., 45., 55., 65., 75.])
Coordinates:
  * depth    (depth) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75.0
Attributes:
    standard_name:          depth
    long_name:              depth from top of sea ice
    units:                  cm
    coverage_content_type:  coordinate
    positive:               down
    bounds:                 depth_bounds 

<xarray.DataArray 'depth_bounds' (depth: 8, bounds: 2)>
array([[ 0, 10],
       [10, 20],
       [20, 30],
       [30, 40],
       [40, 50],
       [50, 60],
       [60, 70],
       [70, 80]])
Coordinates:
  * depth    (depth) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75.0
  * bounds   (bounds) int64 0 1
Attributes:
    long_name:  Minimum and maximum depth bounds in sea ice core
    units:      cm 

<xarray.DataArray 'sea_ice_salinity' (depth: 8)>
array([35.5, 35.6, 35.8, 36. , 36.2, 35.9, 35.8, 35.4])
Coordinates:
  * depth    (depth) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75

# Global attributes

Finally, let's add some global attributes that describe the file as a whole. Here, we follow the ACDD conventions: https://wiki.esipfed.org/Attribute_Convention_for_Data_Discovery_1-3

For datasets that contribute to SIOS, we have more specific requirements based on the ACDD conventions. This is a good list for anyone to use, as the ACDD conventions listed above have not been updated in a while: https://adc.met.no/node/4

In [8]:
xrds.attrs['title'] = 'my title'
dtnow = dt.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
xrds.attrs['date_created'] = dtnow
xrds.attrs['history'] = f'File create at {dtnow} using xarray in Python'

print(xrds)

<xarray.Dataset>
Dimensions:           (depth: 8, bounds: 2)
Coordinates:
  * depth             (depth) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75.0
  * bounds            (bounds) int64 0 1
Data variables:
    sea_ice_salinity  (depth) float64 35.5 35.6 35.8 36.0 36.2 35.9 35.8 35.4
    depth_bounds      (depth, bounds) int64 0 10 10 20 20 30 ... 60 60 70 70 80
Attributes:
    title:         my title
    date_created:  2023-05-03T08:23:56Z
    history:       File create at 2023-05-03T08:23:56Z using xarray in Python


# Encoding and saving the file

In [9]:
myencoding = {
    'depth': {
        'dtype': 'float32',
        '_FillValue': None # Coordinate variables should not have fill values.
        },
    'sea_ice_salinity': {
        'dtype': 'float32',
        '_FillValue': 1e30,
        'zlib': False
        }, 
    'depth_bounds': {
        'dtype': 'int32',
        '_FillValue': 1e6,
        'zlib': False
        },
    'bounds': {
        'dtype': 'int32',
        '_FillValue': None # Coordinate variables should not have fill values.
        }
    }


xrds.to_netcdf('ice_core_salinity_profile.nc',encoding=myencoding)